

Important: in kernel-only competitions we can't use internet connections. So I use pretrained models from here: https://www.kaggle.com/bminixhofer/pytorch-pretrained-image-models

In [ ]:
# libraries
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler
import time 
import tqdm
from PIL import Image
train_on_gpu = True
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
import copy
import cv2
import albumentations
from albumentations import torch as AT

## Data overview

In [ ]:
train = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
test = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
sample_submission = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')

old_train = pd.read_csv('../input/diabetic-retinopathy-resized/trainLabels.csv')

In [ ]:
train.head()

In [ ]:
train['diagnosis'].value_counts().plot(kind='bar');
plt.title('Class counts');

We have a slight disbalance in data.

In [ ]:
"""fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(old_train['level'].unique()):
    for i, (idx, row) in enumerate(old_train.loc[old_train['level'] == class_id].sample(10).iterrows()):
        ax = fig.add_subplot(5, 10, class_id * 10 + i + 1, xticks=[], yticks=[])
        im = Image.open(f'../input/diabetic-retinopathy-resized/resized_train/resized_train/{row["image"]}.jpeg')
        plt.imshow(im)
        ax.set_title(f'Label: {level}')"""

Personally I see little differences between images

## Data preparation

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
def prepare_labels(y):
    # From here: https://www.kaggle.com/pestipeti/keras-cnn-starter
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)

    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

    y = onehot_encoded
    return y, label_encoder

In [ ]:
y, le = prepare_labels(train['diagnosis'])

In [ ]:
def circle_crop(img, sigmaX=10):   
    """
    Create circular crop around image centre    
    """    
    
    img = cv2.imread(img)
    img = crop_image_from_gray(img)    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    height, width, depth = img.shape    
    
    x = int(width/2)
    y = int(height/2)
    r = np.amin((x,y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    img=cv2.addWeighted ( img,4, cv2.GaussianBlur( img , (0,0) , sigmaX) ,-4 ,128)
    return img 

In [ ]:
class GlassDataset(Dataset):
    def __init__(self, df, datatype='train', transform = transforms.Compose([transforms.CenterCrop(32),transforms.ToTensor()]), y = None):
        self.df = df
        self.datatype = datatype
        if self.datatype == 'train':
            self.image_files_list = [f'../input/aptos2019-blindness-detection/{self.datatype}_images/{i}.png' for i in df['id_code'].values]
            self.labels = y
        else:
            self.image_files_list = [f'../input/aptos2019-blindness-detection/{self.datatype}_images/{i}.png' for i in df['id_code'].values]
            self.labels = np.zeros((df.shape[0], 5))
        self.transform = transform

    def __len__(self):
        return len(self.image_files_list)

    def __getitem__(self, idx):
        img_name = self.image_files_list[idx]
        img = cv2.imread(img_name)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        image = self.transform(image=img)
        image = image['image']

        img_name_short = self.image_files_list[idx].split('.')[0]

        label = self.labels[idx]
        if self.datatype == 'test':
            return image, label, img_name
        else:
            return image, label

In [ ]:

data_transforms = albumentations.Compose([
    albumentations.Resize(224, 224),
    albumentations.HorizontalFlip(),
    albumentations.RandomBrightness(),
    albumentations.ShiftScaleRotate(rotate_limit=15, scale_limit=0.10),
    albumentations.JpegCompression(80),
    albumentations.HueSaturationValue(),
    albumentations.Normalize(),
    AT.ToTensor()
    ])
data_transforms_test = albumentations.Compose([
    albumentations.Resize(224, 224),
    albumentations.Normalize(),
    AT.ToTensor()
    ])


dataset = GlassDataset(df=train, datatype='train', transform=data_transforms, y=y)
test_set = GlassDataset(df=test, datatype='test', transform=data_transforms_test)
tr, val = train_test_split(train.diagnosis, stratify=train.diagnosis, test_size=0.1)
train_sampler = SubsetRandomSampler(list(tr.index))
valid_sampler = SubsetRandomSampler(list(val.index))
batch_size = 16
num_workers = 0
# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=num_workers)



In [ ]:
#test_set = GlassDataset(df=test, datatype='test', transform=data_transforms_test)
#test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=num_workers)



In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
import os
os.listdir("../input/pretrained-pytorch-models")

In [ ]:
model_conv = torchvision.models.resnet101()
model_conv.load_state_dict(torch.load('../input/pytorch-pretrained-models/resnet101-5d3b4d8f.pth'))
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(2048, 5)

In [ ]:
"""model_conv2 = torchvision.models.resnet50()
model_conv2.load_state_dict(torch.load('../input/pytorch-pretrained-models/resnet50-19c8e357.pth'))
num_ftrs2 = model_conv2.fc.in_features
model_conv2.fc = nn.Linear(2048, 5)"""

In [ ]:
"""model_conv3 = torchvision.models.vgg16()
model_conv3.load_state_dict(torch.load('../input/vgg16/vgg16.pth'))
model_conv3.classifier[6] = nn.Linear(4096,5)"""

In [ ]:
"""model_conv4 = torchvision.models.densenet121()
model_conv4.load_state_dict(torch.load('../input/pytorch-pretrained-image-models/densenet121.pth'))
model_conv4.classifier = nn.Linear(1024, 5)"""

In [ ]:
"""model_conv5 = torchvision.models.inception_v3()
model_conv5.load_state_dict(torch.load('../input/pretrained-pytorch-models/inception_v3_google-1a9a5a14.pth'))
# Handle the auxilary net
num_ftrs = model_conv5.AuxLogits.fc.in_features
model_conv5.AuxLogits.fc = nn.Linear(num_ftrs, 5)
# Handle the primary net
num_ftrs = model_conv5.fc.in_features
model_conv5.fc = nn.Linear(num_ftrs,5)"""


In [ ]:
model1_path = "../input/aptos1/model_1.pt"
model2_path = "../input/aptos1/model_2.pt"
model3_path = "../input/aptos1/model_3.pt"
model4_path = "../input/aptos1/model_4.pt"

In [ ]:
def train_model(model_conv, train_loader, valid_loader, criterion, optimizer, n_epochs=15, attempt=1):
    model_conv.to(device)
    valid_loss_min = np.Inf
    patience = 5
    # current number of epochs, where validation loss didn't increase
    p = 0
    # whether training should be stopped
    stop = False

    # number of epochs to train the model
    for epoch in range(1, n_epochs+1):
        print(time.ctime(), 'Epoch:', epoch)

        train_loss = []
        train_auc = []

        for batch_i, (data, target) in enumerate(train_loader):

            data, target = data.cuda(), target.cuda()

            optimizer.zero_grad()
            output = model_conv(data)
            loss = criterion(output, target.float())
            train_loss.append(loss.item())

            a = target.data.cpu().numpy()
            b = output[:,-1].detach().cpu().numpy()
            # train_auc.append(roc_auc_score(a, b))
            loss.backward()
            optimizer.step()

        model_conv.eval()
        val_loss = []
        val_auc = []
        for batch_i, (data, target) in enumerate(valid_loader):
            data, target = data.cuda(), target.cuda()
            output = model_conv(data)

            loss = criterion(output, target.float())

            val_loss.append(loss.item()) 
            a = target.data.cpu().numpy()
            b = output[:,-1].detach().cpu().numpy()
            # val_auc.append(roc_auc_score(a, b))

        # print(f'Epoch {epoch}, train loss: {np.mean(train_loss):.4f}, valid loss: {np.mean(val_loss):.4f}, train auc: {np.mean(train_auc):.4f}, valid auc: {np.mean(val_auc):.4f}')
        print(f'Epoch {epoch}, train loss: {np.mean(train_loss):.4f}, valid loss: {np.mean(val_loss):.4f}.')

        valid_loss = np.mean(val_loss)
        scheduler.step(valid_loss)
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model_conv.state_dict(), 'model_{}.pt'.format(attempt))
            valid_loss_min = valid_loss
            p = 0

        # check if validation loss didn't improve
        if valid_loss > valid_loss_min:
            p += 1
            print(f'{p} epochs of increasing val loss')
            if p > patience:
                print('Stopping training')
                stop = True
                break        

        if stop:
            break
    return model_conv

In [ ]:

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model_conv.fc.parameters(), lr=0.01, momentum=0.99)
#scheduler = CyclicLR(optimizer, base_lr=lr, max_lr=0.01, step_size=5, mode='triangular2')
scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.8, patience=2, )

## Model training

In [ ]:
model_resnet101 = train_model(model_conv, train_loader, valid_loader, criterion = criterion, 
                              optimizer = optimizer, n_epochs=15, attempt=1)

In [ ]:
"""model_resnet50 = train_model(model_conv2, train_loader, valid_loader, criterion = criterion, 
                              optimizer = optimizer, n_epochs=15, attempt=2)"""

In [ ]:
"""model_densenet121 = train_model(model_conv4, train_loader, valid_loader, criterion = criterion, 
                              optimizer = optimizer, n_epochs=15, attempt=4)"""

In [ ]:
"""model_vgg16 = train_model(model_conv3, train_loader, valid_loader, criterion = criterion, 
                              optimizer = optimizer, n_epochs=15, attempt=3)"""

In [ ]:
#model_inception = train_model(model_conv5, train_loader_inc, valid_loader_inc, criterion = criterion, 
#                              optimizer = optimizer, n_epochs=15, attempt=5)

## Predicting

In [ ]:
def test_model(model, test_loader):
    sub = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
    model.eval()
    for (data, target, name) in test_loader:
        data = data.cuda()
        output = model(data)
        output = output.cpu().detach().numpy()
        for i, (e, n) in enumerate(list(zip(output, name))):
            sub.loc[sub['id_code'] == n.split('/')[-1].split('.')[0], 'diagnosis'] = le.inverse_transform([np.argmax(e)])
    print( "done")
    return sub

In [ ]:
test_resnet101_1 = test_model(model_resnet101, test_loader)
test_resnet101_2 = test_model(model_resnet101, test_loader)
test_resnet101_3 = test_model(model_resnet101, test_loader)
test_resnet101_4 = test_model(model_resnet101, test_loader)
test_resnet101_5 = test_model(model_resnet101, test_loader)
test_resnet101_6 = test_model(model_resnet101, test_loader)
test_resnet101_7 = test_model(model_resnet101, test_loader)
test_resnet101_8 = test_model(model_resnet101, test_loader)
test_resnet101_9 = test_model(model_resnet101, test_loader)
test_resnet101_10 = test_model(model_resnet101, test_loader)


In [ ]:
test_preds = (test_resnet101_1.diagnosis + test_resnet101_2.diagnosis + test_resnet101_3.diagnosis + test_resnet101_4.diagnosis + test_resnet101_5.diagnosis 
              + test_resnet101_6.diagnosis + test_resnet101_7.diagnosis  
               + test_resnet101_8.diagnosis + test_resnet101_9.diagnosis + test_resnet101_10.diagnosis) / 10

In [ ]:
coef = [0.5, 1.5, 2.5, 3.5]

for i, pred in enumerate(test_preds):
    if pred < coef[0]:
        test_preds[i] = 0
    elif pred >= coef[0] and pred < coef[1]:
        test_preds[i] = 1
    elif pred >= coef[1] and pred < coef[2]:
        test_preds[i] = 2
    elif pred >= coef[2] and pred < coef[3]:
        test_preds[i] = 3
    else:
        test_preds[i] = 4


In [ ]:
sample = pd.read_csv("../input/aptos2019-blindness-detection/sample_submission.csv")
sample.diagnosis = test_preds.astype(int)
sample.to_csv("submission.csv", index=False)

In [ ]:
#test_resnet50 = test_model(model_resnet50, test_loader)

In [ ]:
#test_densenet121 = test_model(model_densenet121, test_loader)

In [ ]:
#test_vgg16 = test_model(model_vgg16, test_loader)

In [ ]:
#test_inception = test_model(model_inception, test_loader_inc)

In [ ]:
"""sub = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')

model_conv.eval()
for (data, target, name) in test_loader:
    data = data.cuda()
    output = model_conv(data)
    output = output.cpu().detach().numpy()
    for i, (e, n) in enumerate(list(zip(output, name))):
        sub.loc[sub['id_code'] == n.split('/')[-1].split('.')[0], 'diagnosis'] = le.inverse_transform([np.argmax(e)])
        
sub.to_csv('submission.csv', index=False)"""

In [ ]:
#sub.head()

In [ ]:
#sub['diagnosis'].value_counts()